In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

from ENCClassifier3 import ENCClassifier
from ENCModelTreeClassifier3 import ENCModelTreeClassifier
from RFwENCMTClassifier3 import RFwENCMTClassifier

In [2]:
warnings.filterwarnings("ignore")

In [3]:
X, y = make_classification(n_samples=2000,n_features=20,n_informative=10,random_state=42)
df = pd.merge(pd.DataFrame(X),pd.DataFrame(y),left_index=True,right_index=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [4]:
def MP(Model, X, y):
    My_scores = Model.predict_proba(X)
    My_predictions = Model.predict(X)
    TN, FP, FN, TP = confusion_matrix(y, My_predictions, labels=[0, 1]).ravel()
    accuracy    = (TP+TN)/(TP+TN+FP+FN)
    sensitivity = TP / (TP+FN)
    specifity   = TN / (TN+FP)
    if My_scores.shape[1]==2:
        fpr, tpr, thresholds = roc_curve(y, My_scores[:,1])
    else:
        fpr, tpr, thresholds = roc_curve(y, My_scores)
    AUC = roc_auc_score(y,My_predictions)
    return AUC, accuracy, sensitivity, specifity

In [5]:
# ENC Classifier
My_model = ENCClassifier()
My_model.fit(X_train,y_train)
print(MP(My_model,X_test,y_test))

(0.5, 0.5016666666666667, 1.0, 0.0)


In [6]:
%%time
# ENC Model Tree Classifier
My_model = ENCModelTreeClassifier(random_state=42)
My_model.fit(X_train,y_train)
print(MP(My_model,X_test,y_test))

(0.8002200024444717, 0.8, 0.7342192691029901, 0.8662207357859532)
CPU times: total: 2.94 s
Wall time: 2.56 s


In [7]:
%%time
# RF w ENCMT Classifier
My_model = RFwENCMTClassifier()
My_model.fit(X_train,y_train)
print(MP(My_model,X_test,y_test))

(0.8933543706041177, 0.8933333333333333, 0.8870431893687708, 0.8996655518394648)
CPU times: total: 25.9 s
Wall time: 26.2 s
